# __Customer Segmentation with RFM Analysis__

## __Business Problem__

FLO, an online shoe store, wants to segment its customers and determine marketing strategies according to these segments. For this purpose, the behaviors of the customers will be defined and groups will be formed according to the clusters in these behaviors.

## __Dataset__

The dataset consists of the information obtained from the past shopping behavior of customers who made their last purchases from Flo as OmniChannel (both online and offline shopper) between 2020 and 2021.

__master_id :__ Unique customer number

__order_channel :__ Which channel of the shopping platform is used (Android, ios, Desktop, Mobile)

__last_order_channel :__ The channel where the most recent purchase was made

__first_order_date :__ Date of the customer's first purchase

__last_order_date :__ Customer's last purchase date

__last_order_date_online :__ The date of the last purchase made by the customer on the online platform

__last_order_date_offline :__ The date of the last purchase made by the customer on the offline platform

__order_num_total_ever_online :__ The total number of purchases made by the customer on the online platform

__order_num_total_ever_offline :__ Total number of purchases made by the customer offline

__customer_value_total_ever_offline :__ Total fee paid by the customer for offline purchases

__customer_value_total_ever_online :__ The total fee paid by the customer for their online shopping

__interested_in_categories_12 :__ List of categories the customer has shopped in the last 12 months

## __Analysis__

### Importing Necesssary Libraries

In [2]:
import datetime as dt
import pandas as pd
import seaborn as sns
pd.options.display.width= None
pd.options.display.max_columns= None
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

### Import Data

In [3]:
df_=pd.read_csv(r"flo_data_20k.csv")
df = df_.copy()
df.head()

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.000,1.000,139.990,799.380,[KADIN]
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.000,2.000,159.970,1853.580,"[ERKEK, COCUK, KADIN, AKTIFSPOR]"
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.000,2.000,189.970,395.350,"[ERKEK, KADIN]"
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.000,1.000,39.990,81.980,"[AKTIFCOCUK, COCUK]"
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.000,1.000,49.990,159.990,[AKTIFSPOR]


### Exploratory Analysis

In [4]:
df.columns

Index(['master_id', 'order_channel', 'last_order_channel', 'first_order_date',
       'last_order_date', 'last_order_date_online', 'last_order_date_offline',
       'order_num_total_ever_online', 'order_num_total_ever_offline',
       'customer_value_total_ever_offline', 'customer_value_total_ever_online',
       'interested_in_categories_12'],
      dtype='object')

In [5]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
order_num_total_ever_online,19945.000,3.111,4.226,1.000,1.000,2.000,4.000,200.000
order_num_total_ever_offline,19945.000,1.914,2.063,1.000,1.000,1.000,2.000,109.000
customer_value_total_ever_offline,19945.000,253.923,301.533,10.000,99.990,179.980,319.970,18119.140
customer_value_total_ever_online,19945.000,497.322,832.602,12.990,149.980,286.460,578.440,45220.130


In [6]:
df.isnull().sum()

master_id                            0
order_channel                        0
last_order_channel                   0
first_order_date                     0
last_order_date                      0
last_order_date_online               0
last_order_date_offline              0
order_num_total_ever_online          0
order_num_total_ever_offline         0
customer_value_total_ever_offline    0
customer_value_total_ever_online     0
interested_in_categories_12          0
dtype: int64

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19945 entries, 0 to 19944
Data columns (total 12 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   master_id                          19945 non-null  object 
 1   order_channel                      19945 non-null  object 
 2   last_order_channel                 19945 non-null  object 
 3   first_order_date                   19945 non-null  object 
 4   last_order_date                    19945 non-null  object 
 5   last_order_date_online             19945 non-null  object 
 6   last_order_date_offline            19945 non-null  object 
 7   order_num_total_ever_online        19945 non-null  float64
 8   order_num_total_ever_offline       19945 non-null  float64
 9   customer_value_total_ever_offline  19945 non-null  float64
 10  customer_value_total_ever_online   19945 non-null  float64
 11  interested_in_categories_12        19945 non-null  obj

In [ ]:
#Find the total value and order of the each customer. In addition, convert date columns to datetime object
df["total_order"]=df["order_num_total_ever_online"]+df.order_num_total_ever_offline
df["monetary"]=df.customer_value_total_ever_offline+df.customer_value_total_ever_online
df.loc[:, df.columns.str.contains("date")]=df.loc[:, df.columns.str.contains("date")]\
                                             .apply(pd.to_datetime,format='%Y-%m-%d')

In [9]:
#Observation of order_channel categories.
##As can be seen from the table, the highest number of customers are registered in the Android App, 
##and similarly, the channel with the highest order and total return is also Android App.
df.groupby("order_channel").agg({"master_id":lambda id: id.nunique(),
                                 "total_order":lambda order: order.sum(),
                                 "monetary":lambda monetary: monetary.sum()})

,master_id,total_order,monetary
order_channel,,,
Android App,9495,52269.000,7819062.760
Desktop,2735,10920.000,1610321.460
Ios App,2833,15351.000,2525999.930
Mobile,4882,21679.000,3028183.160


In [10]:
df[["master_id","monetary"]].sort_values(by="monetary",ascending=False).head(10)

,master_id,monetary
11150,5d1c466a-9cfd-11e9-9897-000d3a38a36f,45905.100
4315,d5ef8058-a5c6-11e9-a2fc-000d3a38a36f,36818.290
7613,73fd19aa-9e37-11e9-9897-000d3a38a36f,33918.100
13880,7137a5c0-7aad-11ea-8f20-000d3a38a36f,31227.410
9055,47a642fe-975b-11eb-8c2a-000d3a38a36f,20706.340
7330,a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,18443.570
8068,d696c654-2633-11ea-8e1c-000d3a38a36f,16918.570
163,fef57ffa-aae6-11e9-a2fc-000d3a38a36f,12726.100
7223,cba59206-9dd1-11e9-9897-000d3a38a36f,12282.240
18767,fc0ce7a4-9d87-11e9-9897-000d3a38a36f,12103.150


In [11]:
df[["master_id","total_order"]].sort_values(by="total_order",ascending=False).head(10)


,master_id,total_order
11150,5d1c466a-9cfd-11e9-9897-000d3a38a36f,202.000
7223,cba59206-9dd1-11e9-9897-000d3a38a36f,131.000
8783,a57f4302-b1a8-11e9-89fa-000d3a38a36f,111.000
2619,fdbe8304-a7ab-11e9-a2fc-000d3a38a36f,88.000
6322,329968c6-a0e2-11e9-a2fc-000d3a38a36f,83.000
7613,73fd19aa-9e37-11e9-9897-000d3a38a36f,82.000
9347,44d032ee-a0d4-11e9-a2fc-000d3a38a36f,77.000
10954,b27e241a-a901-11e9-a2fc-000d3a38a36f,75.000
8068,d696c654-2633-11ea-8e1c-000d3a38a36f,70.000
7330,a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,70.000


In [ ]:
df.last_order_date.max()
#2021-05-30
today_date=dt.datetime(2021,6,2)
df["recency"]=(today_date-df.last_order_date).dt.days
df.rename(columns={"total_order":"frequency"},inplace=True)
rfm=df[["master_id","recency","frequency","monetary"]]
rfm["recency_score"] = pd.qcut(rfm['recency'], 5, labels=[5, 4, 3, 2, 1])
rfm["frequency_score"] = pd.qcut(rfm['frequency'].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])
rfm["monetary_score"] = pd.qcut(rfm['monetary'], 5, labels=[1, 2, 3, 4, 5])
rfm["RF_SCORE"] = (rfm['recency_score'].astype(str) +
                    rfm['frequency_score'].astype(str))


In [13]:
rfm.head()

,master_id,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RF_SCORE
0,cc294636-19f0-11eb-8d74-000d3a38a36f,96,5.000,939.370,3,4,4,34
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,106,21.000,2013.550,3,5,5,35
2,69b69676-1a40-11ea-941b-000d3a38a36f,187,5.000,585.320,2,4,3,24
3,1854e56c-491f-11eb-806e-000d3a38a36f,136,2.000,121.970,3,1,1,31
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,87,2.000,209.980,3,1,1,31


In [ ]:
seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_Risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
}
rfm.loc[:,"RF_SCORE"].replace(seg_map,regex=True,inplace=True)


<img src="image.png">

In [15]:
rfm.head()

,master_id,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RF_SCORE
0,cc294636-19f0-11eb-8d74-000d3a38a36f,96,5.000,939.370,3,4,4,loyal_customers
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,106,21.000,2013.550,3,5,5,loyal_customers
2,69b69676-1a40-11ea-941b-000d3a38a36f,187,5.000,585.320,2,4,3,at_Risk
3,1854e56c-491f-11eb-806e-000d3a38a36f,136,2.000,121.970,3,1,1,about_to_sleep
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,87,2.000,209.980,3,1,1,about_to_sleep


In [19]:
rfm.groupby("RF_SCORE").agg({"recency":"mean",
                            "frequency":"mean",
                            "monetary":"mean"})

,recency,frequency,monetary
RF_SCORE,,,
about_to_sleep,114.785,2.401,359.009
at_Risk,242.607,4.472,646.610
cant_loose,236.444,10.698,1474.468
champions,18.107,8.934,1406.625
hibernating,248.950,2.394,366.267
loyal_customers,83.595,8.375,1216.819
need_attention,114.829,3.728,562.143
new_customers,18.918,2.000,339.956
potential_loyalists,38.156,3.304,533.184


## RFM Case Studies

### Example 1
FLO includes a new women's shoe brand. The product prices of the brand it includes are in the upper segment compared to the preferences of the general customer profile. For this reason, it is desired to contact the customers in the profile that will be interested in the promotion of the brand and product sales. People who shop from loyal customers (champions, loyal customers) and women are the customers to be contacted specifically.

In [17]:
champ_loyal_ids=rfm[(rfm["RF_SCORE"] == "champions")|(rfm["RF_SCORE"] == "loyal_customers")].index
woman_category=df[df["interested_in_categories_12"].astype(str).str.contains("KADIN")].index
womens_shoe=df.loc[(champ_loyal_ids.intersection(woman_category)),"master_id"]

womens_shoe.head()

0     cc294636-19f0-11eb-8d74-000d3a38a36f
1     f431bd5a-ab7b-11e9-a2fc-000d3a38a36f
8     cfbda69e-5b4f-11ea-aca7-000d3a38a36f
11    c2e15af2-9eed-11e9-9897-000d3a38a36f
33    fb840306-1219-11ea-a001-000d3a38a36f
Name: master_id, dtype: object

### Example 2
Nearly 40% discount is planned for Men's and Children's products. It is aimed to specifically target customers who are good customers in the past, but who have not shopped for a long time, who are interested in the categories related to this discount, who should not be lost, those who are asleep and new customers.

In [18]:
rf_score=rfm[(rfm["RF_SCORE"] == "new_customers")|(rfm["RF_SCORE"] == "about_to_sleep")|(rfm["RF_SCORE"] == "cant_loose")].index
category=df[df["interested_in_categories_12"].astype(str).str.contains("ERKEK")|df["interested_in_categories_12"].astype(str).str.contains("ERKEK")].index
target_customer=df.loc[(rf_score.intersection(category)),"master_id"]

target_customer.head()

34     1acf7870-87af-11ea-ace9-000d3a38a36f
56     92a334ee-58f2-11eb-9e65-000d3a38a36f
78     56ddd00a-aff6-11e9-9757-000d3a38a36f
126    71976318-2aa0-11eb-b280-000d3a38a36f
142    d3fea560-aced-11ea-b736-000d3a38a36f
Name: master_id, dtype: object